In [40]:
import pandas as pd
from library import *
from collections import defaultdict

In [72]:
conn = sqlite3.connect('/data/cfb_game_db.sqlite3')

df = pd.read_sql_query("""
                SELECT
                game_thread, team_affected, count(team_affected) as count
                FROM
                calls
                GROUP BY
                game_thread, team_affected
                ORDER BY
                game_thread;
                """, conn)

aggressiveness = pd.read_sql_query("""
                SELECT
                team, CAST(argumentative AS float) / CAST(positive AS float) as agg
                FROM
                fanbase;
                """, conn).fillna(0)

In [42]:
game_dict = get_games_dict()

In [73]:
aggressiveness.head()

,team,agg
0,ohio state buckeyes,0.077341
1,oregon ducks,0.050718
2,florida gators,0.066574
3,east carolina pirates,0.000000
4,ucla bruins,0.040741


In [57]:
calls_for = defaultdict(int)
calls_against = defaultdict(int)
games = defaultdict(int)
groups = df.groupby('game_thread')
for group in groups:
    #print(group)
    teams = game_dict[group[0]]
    #print(type(group[1]))
    for team in teams:
        team = [x for x in team][0]
        games[team] += 1
        for i, row in group[1].iterrows():
            if row['team_affected'] in team:
                calls_against[team] += row['count']
            else:
                calls_for[team] += row['count']
    

In [70]:
# print(calls_against)
# print(calls_for)

In [81]:
keys = set()
for k in calls_for.keys():
    keys.add(k)
for k in calls_against.keys():
    keys.add(k)

differentials = []
scaled = []
output = []
for k in keys:
    differential = calls_for[k] - calls_against[k]
    differentials.append((differential, k))
    scaled.append((k, round(differential/games[k], 3)))
    output.append((k, round(differential/games[k], 3), aggressiveness[aggressiveness['team'] == k]['agg'].values[0]))

In [82]:
#print(sorted(scaled, key=lambda x: x[1]))

In [83]:
#print(output)

[('florida gators', -1.75, 0.066574202496532592), ('mississippi state bulldogs', 3.9, 0.12698412698412698), ('arkansas razorbacks', 0.833, 0.05181347150259067), ('virginia cavaliers', 3.0, 0.050000000000000003), ('oregon state beavers', 2.0, 0.0), ('north dakota state bison', 0.0, 0.25), ('clemson tigers', 0.167, 0.070707070707070704), ('florida state seminoles', -0.5, 0.12386706948640483), ('colorado buffaloes', 2.5, 0.1038961038961039), ('troy trojans', 3.0, 0.0), ('rutgers scarlet knights', 2.0, 0.16666666666666666), ('smu mustangs', 2.0, 0.0), ('wisconsin badgers', -0.143, 0.058524173027989825), ('oregon ducks', -2.68, 0.050717703349282293), ('tennessee volunteers', -1.385, 0.081159420289855067), ('byu cougars', 0.2, 0.011235955056179775), ('vanderbilt commodores', 4.667, 0.083333333333333329), ('cincinnati bearcats', 0.5, 0.0), ('ucf knights', 2.0, 0.032258064516129031), ('louisville cardinals', 0.333, 0.053571428571428568), ('texas tech red raiders', -1.5, 0.046875), ('georgia bu